In [22]:
import pandas as pd
import matplotlib.pyplot as plt

In [23]:
train_df = pd.read_csv('twitter_training.csv')
train_df.columns = ['tweet_id', 'entity', 'sentiment', 'content']
train_df.head()

,tweet_id,entity,sentiment,content
0,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
1,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
2,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
3,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...
4,2401,Borderlands,Positive,im getting into borderlands and i can murder y...


In [24]:
val_df = pd.read_csv('twitter_validation.csv')
val_df.columns = ['tweet_id', 'entity', 'sentiment', 'content']
val_df.head()

,tweet_id,entity,sentiment,content
0,352,Amazon,Neutral,BBC News - Amazon boss Jeff Bezos rejects clai...
1,8312,Microsoft,Negative,@Microsoft Why do I pay for WORD when it funct...
2,4371,CS-GO,Negative,"CSGO matchmaking is so full of closet hacking,..."
3,4433,Google,Neutral,Now the President is slapping Americans in the...
4,6273,FIFA,Negative,Hi @EAHelp I’ve had Madeleine McCann in my cel...


In [25]:
print(train_df['sentiment'].value_counts())

sentiment
Negative      22542
Positive      20831
Neutral       18318
Irrelevant    12990
Name: count, dtype: int64


In [26]:
print(train_df.isnull().sum())

tweet_id       0
entity         0
sentiment      0
content      686
dtype: int64


In [27]:
import re
import torch
from sklearn.preprocessing import LabelEncoder
from collections import Counter

In [28]:
def clean_tweet(tweet):
    if not isinstance(tweet, str): # kiểm tra dữ liệu đầu vào
        return ""
    tweet = tweet.lower() # chuyển về chữ thường 
    tweet = re.sub(r"http\S+", "", tweet)      # remove URLs
    tweet = re.sub(r"@\w+", "", tweet)         # remove mentions
    tweet = re.sub(r"#", "", tweet)            # remove hashtag symbol
    tweet = re.sub(r"[^\w\s]", "", tweet)      # remove punctuation
    tweet = re.sub(r"\s+", " ", tweet).strip() # remove extra spaces
    return tweet

train_df['clean_text'] = train_df['content'].apply(clean_tweet) 
val_df['clean_text'] = val_df['content'].apply(clean_tweet)

In [29]:
label_encoder = LabelEncoder()
train_df['label'] = label_encoder.fit_transform(train_df['sentiment']) # chuyển các nhãn thành số
val_df['label'] = label_encoder.transform(val_df['sentiment'])

In [30]:
tokenized = train_df['clean_text'].apply(lambda x: x.split()) # biến mỗi văn bản thành một danh sách các từ (token hóa)

In [31]:
word_counts = Counter(word for sentence in tokenized for word in sentence) # đếm tần suất từ trong câu 
vocab = {word: idx + 2 for idx, (word, _) in enumerate(word_counts.items())} # ánh xạ từ sang số nguyên 
vocab['<PAD>'] = 0 # các câu ngắn hơn độ dài cố định gán chỉ số 0
vocab['<UNK>'] = 1 # các từ không có trong từ điển gán chỉ số 1

In [32]:
def encode_sentence(sentence, vocab, max_len=32):
    tokens = sentence.split() # tách từ
    ids = [vocab.get(token, vocab['<UNK>']) for token in tokens] # ánh xạ từ sang số nguyên
    if len(ids) < max_len:
        ids += [vocab['<PAD>']] * (max_len - len(ids)) # thêm 0 vào cuối để đủ độ dài 
    else:
        ids = ids[:max_len] # cắt bớt độ dài đến max_len
    return ids

In [33]:
train_df['input_ids'] = train_df['clean_text'].apply(lambda x: encode_sentence(x, vocab)) # biến văn bản thành chuỗi số nguyên 
val_df['input_ids'] = val_df['clean_text'].apply(lambda x: encode_sentence(x, vocab))

train_df có cột input_ids: danh sách các số đại diện từ


train_df['label']: nhãn cảm xúc đã mã hóa

In [34]:
from torch.utils.data import Dataset

class TweetDataset(Dataset):
    def __init__(self, input_ids, labels):
        self.input_ids = input_ids
        self.labels = labels

    def __len__(self): # trả về số lượng mẫu trong dataset dựa vào độ dài của danh sách labels 
        return len(self.labels)

    def __getitem__(self, idx): 
        return {
            'input_ids': torch.tensor(self.input_ids[idx], dtype=torch.long), # danh sách số nguyên đại diện cho câu tại vị trí idx
            'label': torch.tensor(self.labels[idx], dtype=torch.long) # chuyển danh sách thành tensor kiểu long (số nguyên 64-bit)
        }

In [35]:
train_dataset = TweetDataset( # tạo dataset chuẩn PyTorch để đưa vào DataLoader huấn luyện mô hình 
    input_ids=train_df['input_ids'].tolist(),
    labels=train_df['label'].tolist()
)

val_dataset = TweetDataset(
    input_ids=val_df['input_ids'].tolist(),
    labels=val_df['label'].tolist()
)

In [36]:
from torch.utils.data import DataLoader

train_loader = DataLoader(
    train_dataset, # dữ liệu đầu vào 
    batch_size=32, # mỗi batch chứa 32 dữ liệu ( batch là tập con của dữ liệu đầu vào, được đưa vào mô hình cùng một lúc, thay vì từng mẫu một)
    shuffle=True # trộn ngẫu nhiên dữ liệu sau mỗi epoch(1 vòng huấn luyện dữ liệu) giúp giảm overfitting 
)

val_loader = DataLoader(
    val_dataset,
    batch_size=32,
    shuffle=False
)

In [37]:
batch = next(iter(train_loader)) # lấy batch đầu tiên trong DataLoader
print("Input IDs:", batch['input_ids'].shape) # In ra shape (kích thước) của tensor input_ids trong batch ( sẽ là batch_size, max_len)
print("Labels:", batch['label'].shape) # có 32 nhãn (1 nhãn cho mỗi câu trong batch)      

Input IDs: torch.Size([32, 32])
Labels: torch.Size([32])


In [ ]:
import torch.nn as nn

class LSTMClassifier(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim):
        super(LSTMClassifier, self).__init__()

        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        embedded = self.embedding(x)              
        lstm_out, _ = self.lstm(embedded)  
        last_hidden = lstm_out[:, -1, :]          
        output = self.fc(last_hidden)            
        return output


In [ ]:
vocab_size = len(vocab)
embedding_dim = 100       
hidden_dim = 128
output_dim = len(label_encoder.classes_)  

model = LSTMClassifier(vocab_size, embedding_dim, hidden_dim, output_dim)


In [40]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [41]:
def train(model, dataloader, optimizer, criterion):
    model.train()
    total_loss, total_correct = 0, 0

    for batch in dataloader:
        input_ids = batch['input_ids'].to(device)
        labels = batch['label'].to(device)

        outputs = model(input_ids)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        preds = outputs.argmax(dim=1)
        total_correct += (preds == labels).sum().item()

    accuracy = total_correct / len(dataloader.dataset)
    return total_loss / len(dataloader), accuracy


def evaluate(model, dataloader, criterion):
    model.eval()
    total_loss, total_correct = 0, 0

    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch['input_ids'].to(device)
            labels = batch['label'].to(device)

            outputs = model(input_ids)
            loss = criterion(outputs, labels)

            total_loss += loss.item()
            preds = outputs.argmax(dim=1)
            total_correct += (preds == labels).sum().item()

    accuracy = total_correct / len(dataloader.dataset)
    return total_loss / len(dataloader), accuracy


In [42]:
num_epochs = 10

for epoch in range(num_epochs):
    train_loss, train_acc = train(model, train_loader, optimizer, criterion)
    val_loss, val_acc = evaluate(model, val_loader, criterion)

    print(f"Epoch {epoch+1}/{num_epochs}")
    print(f"Train Loss: {train_loss:.4f}, Accuracy: {train_acc:.4f}")
    print(f"Val   Loss: {val_loss:.4f}, Accuracy: {val_acc:.4f}")


Epoch 1/10
Train Loss: 1.2820, Accuracy: 0.4364
Val   Loss: 1.1609, Accuracy: 0.5666
Epoch 2/10
Train Loss: 1.1403, Accuracy: 0.5935
Val   Loss: 1.0780, Accuracy: 0.6567
Epoch 3/10
Train Loss: 1.0651, Accuracy: 0.6717
Val   Loss: 0.9982, Accuracy: 0.7427
Epoch 4/10
Train Loss: 1.0075, Accuracy: 0.7322
Val   Loss: 0.9526, Accuracy: 0.7848
Epoch 5/10
Train Loss: 0.9651, Accuracy: 0.7752
Val   Loss: 0.9185, Accuracy: 0.8218
Epoch 6/10
Train Loss: 0.9373, Accuracy: 0.8039
Val   Loss: 0.9243, Accuracy: 0.8148
Epoch 7/10
Train Loss: 0.9154, Accuracy: 0.8258
Val   Loss: 0.8999, Accuracy: 0.8378
Epoch 8/10
Train Loss: 0.9004, Accuracy: 0.8407
Val   Loss: 0.8816, Accuracy: 0.8579
Epoch 9/10
Train Loss: 0.8874, Accuracy: 0.8538
Val   Loss: 0.8732, Accuracy: 0.8639
Epoch 10/10
Train Loss: 0.8781, Accuracy: 0.8634
Val   Loss: 0.8778, Accuracy: 0.8579
